In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [3]:
df=pd.read_csv("./Question_level.csv")
df

,Unnamed: 0,n_beginner,n_intermediate,n_advance,accuracy_beginner,accuracy_intermediate,accuracy_advance
0,0,4,0,6,0.001993,1.000000,0.753460
1,1,3,4,3,0.585567,0.716942,0.278396
2,2,2,0,8,0.578313,1.000000,0.061586
3,3,6,4,0,0.046100,0.707588,1.000000
4,4,1,4,5,0.095613,0.667447,0.448442
...,...,...,...,...,...,...,...
135163,61,1,2,7,0.408739,0.592968,0.290533
135164,62,7,3,0,0.688674,0.902457,1.000000
135165,63,0,1,9,1.000000,0.555652,0.784169
135166,64,6,2,2,0.794666,0.639777,0.308068


In [5]:
df=df.drop("Unnamed: 0",axis=1)
df

,n_beginner,n_intermediate,n_advance,accuracy_beginner,accuracy_intermediate,accuracy_advance
0,4,0,6,0.001993,1.000000,0.753460
1,3,4,3,0.585567,0.716942,0.278396
2,2,0,8,0.578313,1.000000,0.061586
3,6,4,0,0.046100,0.707588,1.000000
4,1,4,5,0.095613,0.667447,0.448442
...,...,...,...,...,...,...
135163,1,2,7,0.408739,0.592968,0.290533
135164,7,3,0,0.688674,0.902457,1.000000
135165,0,1,9,1.000000,0.555652,0.784169
135166,6,2,2,0.794666,0.639777,0.308068


In [7]:
#decide prev level
def decide_prev_level(current_level):
    index=df.columns.get_loc(current_level)
    previous_level_index=0
    if index==0:
        previous_level_index=index
    else:
        previous_level_index=index-1
    return previous_level_index

#decide next level
def decide__next_level(current_level):
    index=df.columns.get_loc(current_level)
    next_level_index=0
    if index==2:
        next_level_index=index
    else:
        next_level_index=index+1
    return next_level_index
    

#update no. of question in level
def update_n(df,beg_n,beg_percent,int_n,int_percent,adv_n,adv_percent,new_beginner,new_intermediate,new_advance):
    
    #####################################################################
    # for beginner : level switch - upgrade since no prev : for acc_beg>0
                     # level switch - degrade since no prev : for acc_beg<0
    #######################################################################
    beg_new=(-1) * round(beg_percent*beg_n)
    diff=round(beg_percent*beg_n)
    n_int_beg=diff
    
    
    #################################################################
    # for intermediate : level switch - upgrade or INC : depends on weight
    ##########################################################################

    int_new=int_n

    if int_percent > 0:
        #level switch - upgrade
        int_new_LSU=(-1) * round(int_percent*int_n)
        diff=round(int_percent*int_n)
        n_adv_int=diff
    
        #INC
        int_new_INC=round(int_percent*int_n)
        diff=round(int_percent*int_n)
        if(diff>beg_n):
            while diff>beg_n:
                int_new_INC-=1
                diff-=1
        n_beg_int=(-1) * diff
    
        #deciding weightage : Beginner : 28% , Intermediate : 34% , Advance : 38%
        weight_LSU= (int_new_LSU * 0.34) + (n_adv_int * 0.38)
        weight_INC= (int_new_INC * 0.34) + (n_beg_int * 0.28)
    
        if(weight_LSU > weight_INC):
            int_new=int_new_LSU
            n_adv_int=n_adv_int
            n_beg_int=0
        else:
            int_new=int_new_INC
            n_adv_int=0
            n_beg_int=n_beg_int
    else:
        #DEC
        int_new=round(int_percent*int_n)
        diff=round(int_percent*int_n)
        n_beg_int=(-1) * diff
        n_adv_int=0
    
    #############################################################################
    # for advance : INC since no next
    #############################################################################
    if adv_percent > 0:
        #INC
        adv_new=round(adv_percent*adv_n)
        diff=round(adv_percent*adv_n)
        if(diff>int_n):
            while diff>int_n:
                adv_new-=1
                diff-=1
        n_int_adv=(-1) * diff
    else:
        #DEC
        adv_new=round(adv_percent*adv_n)
        diff=round(adv_percent*adv_n)
        n_int_adv=(-1) * diff

    ###########################################################################
    # Updating values
    ###########################################################################

    # print(beg_n+beg_new + n_beg_int)
    # print(int_n+int_new + n_int_beg + n_int_adv)
    # print(adv_n+adv_new + n_adv_int)
    new_beginner.append(beg_n+beg_new + n_beg_int)
    new_intermediate.append(int_n+int_new + n_int_beg + n_int_adv)
    new_advance.append(adv_n+adv_new + n_adv_int)
      
                   
    
# adding column of percentage increase/decrease in each level
def ranges(accuracy,n):
    updated=0.0

    ########### level decrease ##########
    if accuracy>=0.00 and accuracy<0.10:
        updated= -0.72
    elif accuracy>=0.10 and accuracy<0.20:
        updated= -0.66
    elif accuracy>=0.20 and accuracy<0.30:
        updated= -0.60
    elif accuracy>=0.30 and accuracy<0.40:
        updated= -0.54
    elif accuracy>=0.40 and accuracy<0.45:
        updated= -0.48
    elif accuracy>=0.45 and accuracy<0.50:
        updated= -0.44
    elif accuracy>=0.50 and accuracy<0.55:
        updated= -0.40
    elif accuracy>=0.55 and accuracy<0.60:
        updated= -0.36

    ######### level increase ###############
    elif accuracy>=0.60 and accuracy<0.70:
        updated=0.35
    elif accuracy>=0.70 and accuracy<0.80:
        updated=0.44
    elif accuracy>=0.80 and accuracy<0.85:
        updated=0.51
    elif accuracy>=0.85 and accuracy<0.90:
        updated=0.58
    elif accuracy>=0.90 and accuracy<0.95:
        updated=0.65
    else:
        updated=0.75
    return updated

In [9]:
new_beginner=[]
new_intermediate=[]
new_advance=[]
for i in range(0,len(df)):
    new_beginner.append(ranges(df["accuracy_beginner"][i],df["n_beginner"][i]))
    new_intermediate.append(ranges(df["accuracy_intermediate"][i],df["n_intermediate"][i]))
    new_advance.append(ranges(df["accuracy_advance"][i],df["n_advance"][i]))
    
df["%_beginner"]=new_beginner
df["%_intermediate"]=new_intermediate
df["%_advance"]=new_advance
df

,n_beginner,n_intermediate,n_advance,accuracy_beginner,accuracy_intermediate,accuracy_advance,%_beginner,%_intermediate,%_advance
0,4,0,6,0.001993,1.000000,0.753460,-0.72,0.75,0.44
1,3,4,3,0.585567,0.716942,0.278396,-0.36,0.44,-0.60
2,2,0,8,0.578313,1.000000,0.061586,-0.36,0.75,-0.72
3,6,4,0,0.046100,0.707588,1.000000,-0.72,0.44,0.75
4,1,4,5,0.095613,0.667447,0.448442,-0.72,0.35,-0.48
...,...,...,...,...,...,...,...,...,...
135163,1,2,7,0.408739,0.592968,0.290533,-0.48,-0.36,-0.60
135164,7,3,0,0.688674,0.902457,1.000000,0.35,0.65,0.75
135165,0,1,9,1.000000,0.555652,0.784169,0.75,-0.36,0.44
135166,6,2,2,0.794666,0.639777,0.308068,0.44,0.35,-0.54


In [11]:
new_beginner=[]
new_intermediate=[]
new_advance=[]
for i in range(0,len(df)):
    update_n(df,df["n_beginner"][i],df["%_beginner"][i],df["n_intermediate"][i],df["%_intermediate"][i],df["n_advance"][i],df["%_advance"][i],new_beginner,new_intermediate,new_advance)
df["new_beginner"]=new_beginner
df["new_intermediate"]=new_intermediate
df["new_advance"]=new_advance

In [12]:
df

,n_beginner,n_intermediate,n_advance,accuracy_beginner,accuracy_intermediate,accuracy_advance,%_beginner,%_intermediate,%_advance,new_beginner,new_intermediate,new_advance
0,4,0,6,0.001993,1.000000,0.753460,-0.72,0.75,0.44,7,-3,6
1,3,4,3,0.585567,0.716942,0.278396,-0.36,0.44,-0.60,2,7,1
2,2,0,8,0.578313,1.000000,0.061586,-0.36,0.75,-0.72,3,5,2
3,6,4,0,0.046100,0.707588,1.000000,-0.72,0.44,0.75,8,2,0
4,1,4,5,0.095613,0.667447,0.448442,-0.72,0.35,-0.48,1,6,3
...,...,...,...,...,...,...,...,...,...,...,...,...
135163,1,2,7,0.408739,0.592968,0.290533,-0.48,-0.36,-0.60,2,5,3
135164,7,3,0,0.688674,0.902457,1.000000,0.35,0.65,0.75,3,7,0
135165,0,1,9,1.000000,0.555652,0.784169,0.75,-0.36,0.44,0,0,10
135166,6,2,2,0.794666,0.639777,0.308068,0.44,0.35,-0.54,2,7,1


In [13]:
tempdf=df.copy()

In [14]:
tempdf = tempdf[tempdf['new_beginner'] >= 0]
tempdf = tempdf[tempdf['new_intermediate'] >= 0]
tempdf = tempdf[tempdf['new_advance'] >= 0]
tempdf


,n_beginner,n_intermediate,n_advance,accuracy_beginner,accuracy_intermediate,accuracy_advance,%_beginner,%_intermediate,%_advance,new_beginner,new_intermediate,new_advance
1,3,4,3,0.585567,0.716942,0.278396,-0.36,0.44,-0.60,2,7,1
2,2,0,8,0.578313,1.000000,0.061586,-0.36,0.75,-0.72,3,5,2
3,6,4,0,0.046100,0.707588,1.000000,-0.72,0.44,0.75,8,2,0
4,1,4,5,0.095613,0.667447,0.448442,-0.72,0.35,-0.48,1,6,3
5,7,1,2,0.669704,0.408965,0.747877,0.35,-0.48,0.44,5,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...
135163,1,2,7,0.408739,0.592968,0.290533,-0.48,-0.36,-0.60,2,5,3
135164,7,3,0,0.688674,0.902457,1.000000,0.35,0.65,0.75,3,7,0
135165,0,1,9,1.000000,0.555652,0.784169,0.75,-0.36,0.44,0,0,10
135166,6,2,2,0.794666,0.639777,0.308068,0.44,0.35,-0.54,2,7,1


In [18]:
tempdf['sum_cols'] = tempdf['new_beginner'] + tempdf['new_intermediate'] + tempdf['new_advance']
tempdf

,n_beginner,n_intermediate,n_advance,accuracy_beginner,accuracy_intermediate,accuracy_advance,%_beginner,%_intermediate,%_advance,new_beginner,new_intermediate,new_advance,sum_cols
1,3,4,3,0.585567,0.716942,0.278396,-0.36,0.44,-0.60,2,7,1,10
2,2,0,8,0.578313,1.000000,0.061586,-0.36,0.75,-0.72,3,5,2,10
3,6,4,0,0.046100,0.707588,1.000000,-0.72,0.44,0.75,8,2,0,10
4,1,4,5,0.095613,0.667447,0.448442,-0.72,0.35,-0.48,1,6,3,10
5,7,1,2,0.669704,0.408965,0.747877,0.35,-0.48,0.44,5,2,3,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
135163,1,2,7,0.408739,0.592968,0.290533,-0.48,-0.36,-0.60,2,5,3,10
135164,7,3,0,0.688674,0.902457,1.000000,0.35,0.65,0.75,3,7,0,10
135165,0,1,9,1.000000,0.555652,0.784169,0.75,-0.36,0.44,0,0,10,10
135166,6,2,2,0.794666,0.639777,0.308068,0.44,0.35,-0.54,2,7,1,10


In [21]:
tempdf = tempdf[tempdf['sum_cols'] == 10]
tempdf

,n_beginner,n_intermediate,n_advance,accuracy_beginner,accuracy_intermediate,accuracy_advance,%_beginner,%_intermediate,%_advance,new_beginner,new_intermediate,new_advance,sum_cols
1,3,4,3,0.585567,0.716942,0.278396,-0.36,0.44,-0.60,2,7,1,10
2,2,0,8,0.578313,1.000000,0.061586,-0.36,0.75,-0.72,3,5,2,10
3,6,4,0,0.046100,0.707588,1.000000,-0.72,0.44,0.75,8,2,0,10
4,1,4,5,0.095613,0.667447,0.448442,-0.72,0.35,-0.48,1,6,3,10
5,7,1,2,0.669704,0.408965,0.747877,0.35,-0.48,0.44,5,2,3,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
135163,1,2,7,0.408739,0.592968,0.290533,-0.48,-0.36,-0.60,2,5,3,10
135164,7,3,0,0.688674,0.902457,1.000000,0.35,0.65,0.75,3,7,0,10
135165,0,1,9,1.000000,0.555652,0.784169,0.75,-0.36,0.44,0,0,10,10
135166,6,2,2,0.794666,0.639777,0.308068,0.44,0.35,-0.54,2,7,1,10


In [23]:
tempdf=tempdf.drop("sum_cols",axis=1)
tempdf

,n_beginner,n_intermediate,n_advance,accuracy_beginner,accuracy_intermediate,accuracy_advance,%_beginner,%_intermediate,%_advance,new_beginner,new_intermediate,new_advance
1,3,4,3,0.585567,0.716942,0.278396,-0.36,0.44,-0.60,2,7,1
2,2,0,8,0.578313,1.000000,0.061586,-0.36,0.75,-0.72,3,5,2
3,6,4,0,0.046100,0.707588,1.000000,-0.72,0.44,0.75,8,2,0
4,1,4,5,0.095613,0.667447,0.448442,-0.72,0.35,-0.48,1,6,3
5,7,1,2,0.669704,0.408965,0.747877,0.35,-0.48,0.44,5,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...
135163,1,2,7,0.408739,0.592968,0.290533,-0.48,-0.36,-0.60,2,5,3
135164,7,3,0,0.688674,0.902457,1.000000,0.35,0.65,0.75,3,7,0
135165,0,1,9,1.000000,0.555652,0.784169,0.75,-0.36,0.44,0,0,10
135166,6,2,2,0.794666,0.639777,0.308068,0.44,0.35,-0.54,2,7,1


In [27]:
tempdf.to_csv("final_dataset.csv")